In [7]:
! python notebooks/pytorch_example/main.py --cuda --data notebooks/pytorch_example/data/wikitext-2

| epoch   1 |   200/ 2983 batches | lr 20.00 | ms/batch 11.81 | loss  7.63 | ppl  2056.99
| epoch   1 |   400/ 2983 batches | lr 20.00 | ms/batch 11.51 | loss  6.85 | ppl   945.53
| epoch   1 |   600/ 2983 batches | lr 20.00 | ms/batch 11.48 | loss  6.48 | ppl   654.36
| epoch   1 |   800/ 2983 batches | lr 20.00 | ms/batch 11.49 | loss  6.29 | ppl   539.88
| epoch   1 |  1000/ 2983 batches | lr 20.00 | ms/batch 11.49 | loss  6.15 | ppl   466.47
| epoch   1 |  1200/ 2983 batches | lr 20.00 | ms/batch 11.48 | loss  6.06 | ppl   427.89
| epoch   1 |  1400/ 2983 batches | lr 20.00 | ms/batch 11.51 | loss  5.95 | ppl   384.62
| epoch   1 |  1600/ 2983 batches | lr 20.00 | ms/batch 11.50 | loss  5.94 | ppl   380.10
| epoch   1 |  1800/ 2983 batches | lr 20.00 | ms/batch 11.51 | loss  5.79 | ppl   327.88
| epoch   1 |  2000/ 2983 batches | lr 20.00 | ms/batch 11.53 | loss  5.77 | ppl   321.24
| epoch   1 |  2200/ 2983 batches | lr 20.00 | ms/batch 11.53 | loss  5.66 | ppl   286.07
| epoch   

In [9]:
! python notebooks/pytorch_example/generate.py --cuda --data notebooks/pytorch_example/data/wikitext-2

| Generated 0/1000 words
| Generated 100/1000 words
| Generated 200/1000 words
| Generated 300/1000 words
| Generated 400/1000 words
| Generated 500/1000 words
| Generated 600/1000 words
| Generated 700/1000 words
| Generated 800/1000 words
| Generated 900/1000 words


In [10]:
import os
os.chdir('notebooks')

In [19]:
! python pytorch_example/generate.py --cuda --data pytorch_example/data/wikitext-2

| Generated 0/1000 words


In [25]:
os.chdir('pytorch_example')

In [36]:
! python generate.py --cuda --data data/wikitext-2
# running without that temprature thing

| Generated 0/1000 words


In [26]:
import torch

import data

In [100]:
torch.manual_seed(1111)
device = torch.device("cuda")

with open('model.pt', 'rb') as f:
    model = torch.load(f).to(device)
model.eval()

corpus = data.Corpus('data/wikitext-2')
ntokens = len(corpus.dictionary)

In [107]:
hidden = model.init_hidden(1)
input = torch.randint(ntokens, (1, 1), dtype=torch.long).to(device)
#     input_wsc = 'The city councilmen refused the demonstrators a permit because The city councilmen feared violence.'
# input_wsc = 'The city people refused the demonstrators a permit because The city people feared violence .'
input_wsc = "Joan made sure to thank Susan for all the help Joan had received ."
input_wsc_words = input_wsc.split()
input.fill_(corpus.dictionary.word2idx[input_wsc_words[0]])

number_of_words = len(input_wsc_words)

with open('wsc_generated_probs.txt', 'w') as outf:
    with torch.no_grad():  # no tracking history
        for i in range(number_of_words-1):
            output, hidden = model(input, hidden)
            # TODO shouldn't be using all these things below
            word_weights = output.squeeze().cpu()
#             .div(args.temperature).exp()
#             if not args.wsc:
#                 word_idx = torch.multinomial(word_weights, 1)[0]
#                 input.fill_(word_idx)
#                 word = corpus.dictionary.idx2word[word_idx]
#                 outf.write(word + ('\n' if i % 20 == 19 else ' '))
#             else:
            if i+1 < number_of_words:
                input.fill_(corpus.dictionary.word2idx[input_wsc_words[i+1]])
                outf.write(str(word_weights[corpus.dictionary.word2idx[input_wsc_words[i+1]]]) + ('\n' if i % 20 == 19 else ' '))
#             print()
#             print(str(word_weights[corpus.dictionary.word2idx[input_wsc_words[i]]]))
            

In [90]:
input_wsc_words[i]

'violence'

In [91]:
tens = word_weights[corpus.dictionary.word2idx[input_wsc_words[i+1]]]

In [92]:
tens.float()

tensor(1686347.7500)

In [98]:
output.squeeze()

tensor([ 4.4055,  4.4613, -1.4697,  ..., -2.6670, -0.5011, -1.4255],
       device='cuda:0')

In [96]:
output.size()

torch.Size([1, 1, 33278])

In [99]:
output.squeeze().size()

torch.Size([33278])

In [3]:
import tensorflow as tf
import tensorflow.contrib.slim as slim

In [5]:
sess = tf.Session()
saver = tf.train.import_meta_graph('../../models/research/lm_commonsense/reproduce/lm02/ckpt-best.meta')
saver.restore(sess, '../../models/research/lm_commonsense/reproduce/lm01/ckpt-best')

INFO:tensorflow:Restoring parameters from ../../models/research/lm_commonsense/reproduce/lm01/ckpt-best


In [6]:
def model_summary():
    model_vars = tf.trainable_variables()
    slim.model_analyzer.analyze_vars(model_vars, print_info=True)

model_summary()

---------
Variables: name (type shape) [size]
---------
SotaRNN/embedding/W_0:0 (float32_ref 25000x1024) [25600000, bytes: 102400000]
SotaRNN/embedding/W_1:0 (float32_ref 25000x1024) [25600000, bytes: 102400000]
SotaRNN/embedding/W_2:0 (float32_ref 25000x1024) [25600000, bytes: 102400000]
SotaRNN/embedding/W_3:0 (float32_ref 25000x1024) [25600000, bytes: 102400000]
SotaRNN/embedding/W_4:0 (float32_ref 25000x1024) [25600000, bytes: 102400000]
SotaRNN/embedding/W_5:0 (float32_ref 25000x1024) [25600000, bytes: 102400000]
SotaRNN/embedding/W_6:0 (float32_ref 25000x1024) [25600000, bytes: 102400000]
SotaRNN/embedding/W_7:0 (float32_ref 25000x1024) [25600000, bytes: 102400000]
SotaRNN/embedding/W_8:0 (float32_ref 25000x1024) [25600000, bytes: 102400000]
SotaRNN/embedding/W_9:0 (float32_ref 25000x1024) [25600000, bytes: 102400000]
SotaRNN/embedding/W_10:0 (float32_ref 25000x1024) [25600000, bytes: 102400000]
SotaRNN/embedding/W_11:0 (float32_ref 25000x1024) [25600000, bytes: 102400000]
SotaRN

In [8]:
slim.model_analyzer.analyze_ops(tf.get_default_graph(),
  print_info=True)

---------
Operations: name -> (type shapes) [size]
---------
train_inputs_in 	-> (int32 128x20) [2560]
train_char_inputs_in 	-> (int32 128x20x50) [128000]
Const 	-> (bool ) [1]
train_targets_in 	-> (int32 128x20) [2560]
Placeholder 	-> (float32 128x20) [2560]
eval_inputs_in 	-> (int32 128x20) [2560]
eval_char_inputs_in 	-> (int32 128x20x50) [128000]
Const_1 	-> (bool ) [1]
eval_targets_in 	-> (int32 128x20) [2560]
Placeholder_1 	-> (float32 128x20) [2560]
test_inputs_in 	-> (int32 128x20) [2560]
test_char_inputs_in 	-> (int32 128x20x50) [128000]
test_targets_in 	-> (int32 128x20) [2560]
Placeholder_2 	-> (float32 128x20) [2560]
SotaRNN/zeros/shape_as_tensor 	-> (int32 2) [2]
SotaRNN/zeros/Const 	-> (float32 ) [1]
SotaRNN/zeros 	-> (float32 128x9216) [1179648]
SotaRNN/Variable 	-> (float32_ref 128x9216) [1179648]
SotaRNN/Variable/Assign 	-> (float32_ref 128x9216) [1179648]
SotaRNN/Variable/read 	-> (float32 128x9216) [1179648]
SotaRNN/zeros_1/shape_as_tensor 	-> (int32 2) [2]
SotaRNN/ze

137184444745